# Smoke Detection Model

**Author:** Parker Addison  
**Date:** 2022-September – 2022-December

We'll use a model based on either VGG or ResNet to perform binary classification for the presence of smoke in a single image. This model will be instantiated with pre-trained weights on ImageNet, with the final layer replaced with a fully-connected binary-output layer. All weights are unfrozen and the model is trained on a set of imagery for real wildfire locations with a validation set of additional real locations.

Finally, we'll evaluate the performance of our model against a holdout dataset of a real location that was not included in the training nor validation sets.

In [5]:
from pathlib import Path

import src.data
import src.models
import src.utils

## Baseline Real Training

In [9]:
source = 'data/EXAMPLE/transformed/'
batch_size = 64
train_loader = src.data.load_tensors_from_folder(Path(source, 'train'), batch_size=batch_size)
valid_loader = src.data.load_tensors_from_folder(Path(source, 'valid'), batch_size=batch_size)
holdout_loader = src.data.load_tensors_from_folder(Path(source, 'holdout'), batch_size=batch_size)
virtual_loader = src.data.load_tensors_from_folder(Path(source, 'virtual'), batch_size=batch_size)

Loaded from: data/EXAMPLE/transformed/train
	Class balance: 66.1% smoke (366 total images)
Loaded from: data/EXAMPLE/transformed/valid
	Class balance: 48.5% smoke (297 total images)
Loaded from: data/EXAMPLE/transformed/holdout
	Class balance: 50.1% smoke (523 total images)
Loaded from: data/EXAMPLE/transformed/virtual
	Class balance: 50.0% smoke (648 total images)


In [10]:
# backbone = 'vgg16'
backbone = 'resnet18'
# backbone = 'resnet34'
model, device, criterion, optimizer = src.models.initialize_model(backbone)

In [18]:
src.models.train(model, train_loader, valid_loader, criterion, optimizer, epochs=1);

Epoch 1: 100%|██████████| 6/6 [01:16<00:00, 12.69s/batch, train_loss=0.202, loss=0.709, acc=0.589, auc=0.585, f1=0.512, prec=0.604, rec=0.444]


In [19]:
# Evaluate against the real holdout set
print('Holdout')
src.models.evaluate(model, holdout_loader, criterion, log=True);

Holdout
loss: 0.706	acc: 0.509	auc: 0.508	f1: 0.539	prec: 0.508	rec: 0.573


## Training With Virtual Data 

In [ ]:
# backbone = 'vgg16'
backbone = 'resnet18'
# backbone = 'resnet34'
model_virt, device, criterion, optimizer = src.models.initialize_model(backbone)

In [ ]:
src.models.train(model_virt, virtual_loader, valid_loader, criterion, optimizer, epochs=10);

In [ ]:
# Evaluate against the real holdout set
src.models.evaluate(model_virt, valid_loader, criterion, log=True);

## Training with Combined Real + Virtual Data

In [ ]:
# backbone = 'vgg16'
backbone = 'resnet18'
# backbone = 'resnet34'
model_comb, device, criterion, optimizer = src.models.initialize_model(backbone)
combined_loader = src.data.combine([train_loader.dataset, virtual_loader.dataset], batch_size=batch_size)

In [ ]:
src.models.train(model_comb, combined_loader, valid_loader, criterion, optimizer, epochs=10);

In [ ]:
src.models.evaluate(model_comb, holdout_loader, criterion, log=True);